In [28]:
import boto3
import sagemaker
import pandas as pd
import numpy as np
import botocore
from sagemaker import clarify

# # not actually sure how botocore works, copied this from somewhere, if you know how to make a sagemaker session?
# config = botocore.config.Config(region_name='us-east-2')

# low-level service client of the boto3 session
sm = boto3.client(service_name='sagemaker')

sess = sagemaker.Session()
bucket = sess.default_bucket()
role = sagemaker.get_execution_role()
region = sess.boto_region_name
print(region)

us-east-1


In [29]:
clarify_processor = clarify.SageMakerClarifyProcessor(
    role=role,
    instance_count=1,
    instance_type='ml.c5.2xlarge',
    sagemaker_session=sess
)

bias_report_output_path = "s3://blueberries-dsc/blueberries/output_clarify"

INFO:sagemaker.image_uris:Defaulting to the only supported framework/algorithm version: 1.0.
INFO:sagemaker.image_uris:Ignoring unnecessary instance type: None.


In [34]:
import awswrangler as wr

train_data = wr.s3.read_csv("s3://blueberries-dsc/blueberry.csv")
train_data['yield']

0        4476.81146
1        5548.12201
2        6869.77760
3        6880.77590
4        7479.93417
            ...    
15284    7667.83619
15285    3680.56025
15286    4696.44394
15287    6772.93347
15288    5867.99722
Name: yield, Length: 15289, dtype: float64

In [31]:
data_config = clarify.DataConfig(
    s3_data_input_path="s3://blueberries-dsc/blueberry.csv",
    s3_output_path=bias_report_output_path,
    label='yield',
    headers=train_data.columns.to_list(),
    dataset_type='text/csv'
)

In [32]:
bias_config = clarify.BiasConfig(
    label_values_or_threshold=[5000],
    facet_values_or_threshold=[0.1],
    facet_name = 'AverageRainingDays',
)

In [33]:
clarify_processor.run_pre_training_bias(
    data_config = data_config,
    data_bias_config = bias_config,
    methods = ["CI", "DPL"],
)

INFO:sagemaker.clarify:Analysis Config: {'dataset_type': 'text/csv', 'headers': ['id', 'clonesize', 'honeybee', 'bumbles', 'andrena', 'osmia', 'MaxOfUpperTRange', 'MinOfUpperTRange', 'AverageOfUpperTRange', 'MaxOfLowerTRange', 'MinOfLowerTRange', 'AverageOfLowerTRange', 'RainingDays', 'AverageRainingDays', 'fruitset', 'fruitmass', 'seeds', 'yield'], 'label': 'yield', 'label_values_or_threshold': [5000], 'facet': [{'name_or_index': 'AverageRainingDays', 'value_or_threshold': [0.1]}], 'methods': {'report': {'name': 'report', 'title': 'Analysis Report'}, 'pre_training_bias': {'methods': ['CI', 'DPL']}}}
INFO:sagemaker:Creating processing-job with name Clarify-Pretraining-Bias-2023-06-15-05-22-48-650


............................2023-06-15 05:27:25,507 logging.conf not found when configuring logging, using default logging configuration.
2023-06-15 05:27:25,508 Starting SageMaker Clarify Processing job
2023-06-15 05:27:25,508 Analysis config path: /opt/ml/processing/input/config/analysis_config.json
2023-06-15 05:27:25,508 Analysis result path: /opt/ml/processing/output
2023-06-15 05:27:25,508 This host is algo-1.
2023-06-15 05:27:25,508 This host is the leader.
2023-06-15 05:27:25,508 Number of hosts in the cluster is 1.
2023-06-15 05:27:25,510 Running Python / Pandas based analyzer.
2023-06-15 05:27:25,510 Dataset type: text/csv uri: /opt/ml/processing/input/data
2023-06-15 05:27:25,519 Loading dataset...
/usr/local/lib/python3.9/site-packages/analyzer/data_loading/csv_data_loader.py:329: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(df_tmp, ignore_index=True)
2023-06-15 05:27:25